# AI and Human AI Performance

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle
import random
import copy
import pickle
import random
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()

/opt/homebrew/Caskroom/miniforge/base/envs/deep36/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

# ImageNet
##  Load summary fo AI Performance

In [4]:
random.seed(42)

with open('../data/Classification-Summary-ImageNet.pickle', 'rb') as f:
  classification_summary = pickle.load(f)

In [5]:
classification_summary.keys()

dict_keys(['all_gts', 'resnet_wnid', 'knn_wnid', 'emd_wnid', 'chm_wnid', 'resnet_conf', 'knn_conf', 'emd_conf', 'chm_conf', 'resnet_wnid_real', 'knn_wnid_real', 'emd_wnid_real', 'chm_wnid_real', 'resnet_real_conf', 'knn__real_conf', 'emd__real_conf', 'chm__real_conf'])

### Unpacking

In [6]:
real_gt_real = classification_summary['all_gts']

resnet_wnid_real = classification_summary['resnet_wnid_real']
knn_wnid_real    = classification_summary['knn_wnid_real']   
emd_wnid_real    = classification_summary['emd_wnid_real']   
chm_wnid_real    = classification_summary['chm_wnid_real']   

IsM1Correct_real = classification_summary['resnet_wnid_real'] 
IsM2Correct_real = classification_summary['knn_wnid_real']    
IsM3Correct_real = classification_summary['emd_wnid_real']    
IsM4Correct_real = classification_summary['chm_wnid_real']    

M1Conf_real = classification_summary['resnet_real_conf']  
M2Conf_real = classification_summary['knn__real_conf']    
M3Conf_real = classification_summary['emd__real_conf']    
M4Conf_real = classification_summary['chm__real_conf']    

In [7]:
MethodName = ['ResNet', 'KNN', 'EMD', 'CHM']
Methods    = [IsM1Correct_real,IsM2Correct_real,IsM3Correct_real,IsM4Correct_real]
Confs      = [M1Conf_real,M2Conf_real,M3Conf_real,M4Conf_real]

TAnalysis = {}

for N, M, C in zip(MethodName, Methods, Confs):
  TAnalysis[N] = {}
  for T in tqdm(np.arange(0., 1.05, 0.05)):
    TAnalysis[N][round(T, 2)] = np.average([x[0] == (x[1]>=T) for x in zip(M, C)])

100%|██████████| 21/21 [00:00<00:00, 33.11it/s]


In [8]:
TAnalysis[N][round(T, 2)] = np.average([x[0] == (x[1]>=T) for x in zip(M, C)])

In [9]:
def split_data_44(a, b):
  zipped = list(zip(a, b))
  random.shuffle(zipped)
  ra, rb = zip(*zipped)
  return ra[:2000], rb[:2000],  ra[2000:], rb[2000:]

In [10]:
MethodName = ['ResNet', 'KNN', 'EMD', 'CHM']

IsM1Correct_T, M1Conf_T, IsM1Correct_V, M1Conf_V = split_data_44(IsM1Correct_real, M1Conf_real)
IsM2Correct_T, M2Conf_T, IsM2Correct_V, M2Conf_V = split_data_44(IsM2Correct_real, M2Conf_real)
IsM3Correct_T, M3Conf_T, IsM3Correct_V, M3Conf_V = split_data_44(IsM3Correct_real, M3Conf_real)
IsM4Correct_T, M4Conf_T, IsM4Correct_V, M4Conf_V = split_data_44(IsM4Correct_real, M4Conf_real)


Methods_T = [IsM1Correct_T, IsM2Correct_T, IsM3Correct_T, IsM4Correct_T]
Confs_T   = [M1Conf_T, M2Conf_T, M3Conf_T, M4Conf_T]

TAnalysis_T = {}

for N, M, C in zip(MethodName, Methods_T, Confs_T):
  TAnalysis_T[N] = {}
  for T in tqdm(np.arange(0., 1.05, 0.05)):
    TAnalysis_T[N][round(T, 2)] = np.average([x[0] == (x[1]>=T) for x in zip(M, C)])

100%|██████████| 21/21 [00:00<00:00, 867.38it/s]


In [11]:
Methods_V = [IsM1Correct_V, IsM2Correct_V, IsM3Correct_V, IsM4Correct_V]
Confs_V   = [M1Conf_V, M2Conf_V, M3Conf_V, M4Conf_V]

TAnalysis_V = {}

for N, M, C in zip(MethodName, Methods_V, Confs_V):
  TAnalysis_V[N] = {}
  for T in tqdm(np.arange(0., 1.05, 0.05)):
    TAnalysis_V[N][round(T, 2)] = np.average([x[0] == (x[1]>=T) for x in zip(M, C)])

100%|██████████| 21/21 [00:00<00:00, 40.58it/s]


In [12]:
pd.DataFrame(TAnalysis).T.style.background_gradient(axis=1)

,0.0,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1.0
ResNet,0.831442,0.831623,0.833468,0.839209,0.843328,0.849203,0.854898,0.858838,0.861269,0.861539,0.858995,0.852062,0.841167,0.832613,0.816968,0.803507,0.783338,0.750180,0.717833,0.630110,0.173217
KNN,0.821448,0.821448,0.821583,0.833671,0.833671,0.841910,0.855776,0.858703,0.858703,0.859490,0.855664,0.846840,0.820120,0.820120,0.782663,0.782663,0.758576,0.691676,0.691676,0.540294,0.540294
EMD,0.823901,0.823901,0.823992,0.836507,0.836507,0.845264,0.858027,0.860053,0.860053,0.858995,0.854943,0.846412,0.818364,0.818364,0.776382,0.776382,0.751666,0.680240,0.680240,0.522735,0.522735
CHM,0.820502,0.820502,0.820570,0.830475,0.830475,0.837925,0.851184,0.855416,0.855416,0.856654,0.854313,0.846907,0.821943,0.821943,0.787345,0.787345,0.763371,0.698496,0.698496,0.551324,0.551324


In [13]:
pd.DataFrame(TAnalysis_T).T.style.background_gradient(axis=1)

,0.0,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1.0
ResNet,0.832000,0.832000,0.831000,0.838500,0.842500,0.852500,0.857000,0.863000,0.870500,0.870500,0.863000,0.850000,0.841500,0.830000,0.817000,0.800000,0.772000,0.743500,0.708500,0.610500,0.173000
KNN,0.836500,0.836500,0.837500,0.847000,0.847000,0.849500,0.856000,0.859500,0.859500,0.860000,0.855000,0.848000,0.825500,0.825500,0.784500,0.784500,0.762500,0.693500,0.693500,0.533000,0.533000
EMD,0.840000,0.840000,0.840000,0.851000,0.851000,0.859500,0.873000,0.879500,0.879500,0.875500,0.870500,0.850000,0.825500,0.825500,0.774500,0.774500,0.741000,0.667000,0.667000,0.509000,0.509000
CHM,0.832000,0.832000,0.832000,0.844500,0.844500,0.857500,0.867000,0.868500,0.868500,0.863000,0.869000,0.861000,0.834000,0.834000,0.795500,0.795500,0.775500,0.708500,0.708500,0.552500,0.552500


In [14]:
pd.DataFrame(TAnalysis_V).T.style.background_gradient(axis=1)

,0.0,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1.0
ResNet,0.831416,0.831605,0.833585,0.839242,0.843367,0.849048,0.854799,0.858641,0.860833,0.861116,0.858806,0.852159,0.841151,0.832736,0.816967,0.803672,0.783872,0.750495,0.718273,0.631034,0.173227
KNN,0.820738,0.820738,0.820833,0.833043,0.833043,0.841552,0.855766,0.858665,0.858665,0.859466,0.855695,0.846785,0.819866,0.819866,0.782576,0.782576,0.758391,0.691590,0.691590,0.540637,0.540637
EMD,0.823143,0.823143,0.823237,0.835824,0.835824,0.844593,0.857321,0.859136,0.859136,0.858217,0.854210,0.846243,0.818028,0.818028,0.776471,0.776471,0.752169,0.680865,0.680865,0.523383,0.523383
CHM,0.819960,0.819960,0.820031,0.829813,0.829813,0.837003,0.850438,0.854799,0.854799,0.856355,0.853621,0.846243,0.821375,0.821375,0.786960,0.786960,0.762799,0.698025,0.698025,0.551268,0.551268


In [15]:
# Full 44K
pd.DataFrame(TAnalysis).T.style.highlight_max(color = 'yellow', axis = 1);

In [16]:
# 2K Fine-tuning
pd.DataFrame(TAnalysis_T).T.style.highlight_max(color = 'yellow', axis = 1)

,0.0,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1.0
ResNet,0.832000,0.832000,0.831000,0.838500,0.842500,0.852500,0.857000,0.863000,0.870500,0.870500,0.863000,0.850000,0.841500,0.830000,0.817000,0.800000,0.772000,0.743500,0.708500,0.610500,0.173000
KNN,0.836500,0.836500,0.837500,0.847000,0.847000,0.849500,0.856000,0.859500,0.859500,0.860000,0.855000,0.848000,0.825500,0.825500,0.784500,0.784500,0.762500,0.693500,0.693500,0.533000,0.533000
EMD,0.840000,0.840000,0.840000,0.851000,0.851000,0.859500,0.873000,0.879500,0.879500,0.875500,0.870500,0.850000,0.825500,0.825500,0.774500,0.774500,0.741000,0.667000,0.667000,0.509000,0.509000
CHM,0.832000,0.832000,0.832000,0.844500,0.844500,0.857500,0.867000,0.868500,0.868500,0.863000,0.869000,0.861000,0.834000,0.834000,0.795500,0.795500,0.775500,0.708500,0.708500,0.552500,0.552500


In [17]:
# 42K Test PREVIEW
pd.DataFrame(TAnalysis_V).T.loc[:, 0.30:0.5]*100

,0.30,0.35,0.40,0.45,0.50
ResNet,85.479917,85.864134,86.083349,86.111635,85.880634
KNN,85.576560,85.866491,85.866491,85.946634,85.569489
EMD,85.732133,85.913634,85.913634,85.821705,85.420988
CHM,85.043843,85.479917,85.479917,85.635489,85.362059


In [18]:
optimal_t = pd.DataFrame(TAnalysis_T).T.idxmax(axis=1)

In [19]:
tdf = pd.DataFrame(TAnalysis_T).T
max_indices = np.where(tdf == tdf.max(axis=1)[:, np.newaxis], tdf.columns, None)
max_indices = np.where(max_indices == None, -1, max_indices)
max_indices =  np.amax(max_indices, axis=1)
max_indices

/var/folders/gs/ftfl94c117q3mq9rqwfnx1mc0000gn/T/ipykernel_53457/3501961810.py:2: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  max_indices = np.where(tdf == tdf.max(axis=1)[:, np.newaxis], tdf.columns, None)


array([0.45, 0.45, 0.4, 0.5], dtype=object)

In [20]:
optimal_t

ResNet    0.40
KNN       0.45
EMD       0.35
CHM       0.50
dtype: float64

In [21]:
# clone optimal_t but with values of max_indices
optima_t_max = optimal_t.copy()
optima_t_max[:] = max_indices
optima_t_max

ResNet    0.45
KNN       0.45
EMD        0.4
CHM        0.5
dtype: object

In [22]:
optimal_t = optima_t_max

In [23]:
df = pd.DataFrame(TAnalysis_V).loc[optimal_t, :]
# pick diagonal
ai_performance_values = df.to_numpy()[np.arange(4), np.arange(4)]*100

In [24]:
ai_performance_df = pd.DataFrame(ai_performance_values, index=optimal_t.index, columns=['AI Performance'])

## Final Table for AI Performance

In [25]:
ai_performance_df.round(2) # Final Table for AI Performance

,AI Performance
ResNet,86.11
KNN,85.95
EMD,85.91
CHM,85.36


# Human-AI Team Performance

###  Load summary fo AI Performance

In [26]:
random.seed(42)

with open("../data/Classification-Summary-ImageNet.pickle", "rb") as f:
    classification_summary = pickle.load(f)

In [27]:
classification_summary.keys()

dict_keys(['all_gts', 'resnet_wnid', 'knn_wnid', 'emd_wnid', 'chm_wnid', 'resnet_conf', 'knn_conf', 'emd_conf', 'chm_conf', 'resnet_wnid_real', 'knn_wnid_real', 'emd_wnid_real', 'chm_wnid_real', 'resnet_real_conf', 'knn__real_conf', 'emd__real_conf', 'chm__real_conf'])

### Unpacking

In [28]:
real_gt_real = classification_summary["all_gts"]

resnet_wnid_real = classification_summary["resnet_wnid_real"]
knn_wnid_real = classification_summary["knn_wnid_real"]
emd_wnid_real = classification_summary["emd_wnid_real"]
chm_wnid_real = classification_summary["chm_wnid_real"]

IsM1Correct_real = np.asarray(classification_summary["resnet_wnid_real"])
IsM2Correct_real = np.asarray(classification_summary["knn_wnid_real"])
IsM3Correct_real = np.asarray(classification_summary["emd_wnid_real"])
IsM4Correct_real = np.asarray(classification_summary["chm_wnid_real"])

M1Conf_real = np.asarray(classification_summary["resnet_real_conf"])
M2Conf_real = np.asarray(classification_summary["knn__real_conf"])
M3Conf_real = np.asarray(classification_summary["emd__real_conf"])
M4Conf_real = np.asarray(classification_summary["chm__real_conf"])

## AI Performance Analysis

In [29]:
MethodName = ["ResNet", "KNN", "EMD", "CHM"]
Methods = [IsM1Correct_real, IsM2Correct_real, IsM3Correct_real, IsM4Correct_real]
Confs = [M1Conf_real, M2Conf_real, M3Conf_real, M4Conf_real]

TAnalysis = {}
TCount = {}

for N, M, C in zip(MethodName, Methods, Confs):
    TAnalysis[N] = {}
    TCount[N] = {}

    for T in np.arange(0.0, 1.05, 0.05):
        TAnalysis[N][round(T, 2)] = np.mean(M[C >= T])
        TCount[N][round(T, 2)] = len(M[C >= T])


ai_df = pd.DataFrame(TAnalysis)
print("Done")

Done


## Number of Images 

In [30]:
# Number of Images
count_df = pd.DataFrame(TCount)
count_df

,ResNet,KNN,EMD,CHM
0.00,44424,44424,44424,44424
0.05,44414,44424,44424,44424
0.10,44296,44418,44420,44421
0.15,43863,43651,43618,43773
0.20,43472,43651,43618,43773
0.25,42839,42879,42809,43070
0.30,41934,40823,40806,41253
0.35,41081,39649,39598,40139
0.40,40353,39649,39598,40139
0.45,39317,38354,38317,38914


## Ratios (out of 44424 Images)

In [31]:
# RATIOs (out of 44424 Images)
ratio_df = pd.DataFrame(TCount) / len(real_gt_real)
ratio_df

,ResNet,KNN,EMD,CHM
0.00,1.000000,1.000000,1.000000,1.000000
0.05,0.999775,1.000000,1.000000,1.000000
0.10,0.997119,0.999865,0.999910,0.999932
0.15,0.987372,0.982599,0.981857,0.985346
0.20,0.978570,0.982599,0.981857,0.985346
0.25,0.964321,0.965222,0.963646,0.969521
0.30,0.943949,0.918940,0.918558,0.928620
0.35,0.924748,0.892513,0.891365,0.903543
0.40,0.908360,0.892513,0.891365,0.903543
0.45,0.885040,0.863362,0.862529,0.875968


## Human Performance Analysis

In [32]:
summary_df = pd.read_csv("../data/ImageNet-Human-Study-Summary.csv")

## Removing Bad Users 

We manually marked two users as **bad** based on their average performance on `1:1` ratio samples

1. `6395881` 
1. `6396044`


And after resampling we excluded these `4` users:

1. `6386000`
1. `6382894`
1. `6385998`
1. `6382878`

In [33]:
summary_df = summary_df[
    ~summary_df["UID"].isin([6395881, 6396044] + [6386000, 6382894, 6385998, 6382878])
]

In [34]:
# summary_df.head()

In [35]:
print(f"total {len(summary_df)} entries")

total 10620 entries


In [36]:
df_treshold = copy.deepcopy(summary_df[["Method", "Correctness", "ConfidenceScore"]])

## Human Performance

In [37]:
ORDER_DICT = {
    "ResNet-50": 1,
    "kNN": 2,
    "EMD-NN": 3,
    "EMD-Corr": 4,
    "CHM-NN": 5,
    "CHM-Corr": 6,
}

In [38]:
human_accuracy = {}
for T in np.arange(0.0, 1.05, 0.05):
    human_accuracy[f"{T:0.2f}"] = (
        df_treshold[df_treshold["ConfidenceScore"] < T]
        .groupby("Method")["Correctness"]
        .mean()
    )

In [39]:
humand_df = (
    pd.DataFrame.from_dict(human_accuracy)
    .loc[::-1]
    .reset_index()
    .sort_values(by="Method", key=lambda x: x.map(ORDER_DICT))
    .T
)

humand_df = humand_df.drop("Method")
humand_df.columns = ["ResNet-50", "kNN", "EMD-NN", "EMD-Corr", "CHM-NN", "CHM-Corr"]
humand_df

,ResNet-50,kNN,EMD-NN,EMD-Corr,CHM-NN,CHM-Corr
0.00,NaN,NaN,NaN,NaN,NaN,NaN
0.05,1.0,NaN,NaN,NaN,NaN,NaN
0.10,1.0,1.0,1.0,1.0,NaN,NaN
0.15,0.890909,0.971429,0.961039,0.952941,0.947368,0.918919
0.20,0.859813,0.971429,0.961039,0.952941,0.947368,0.918919
0.25,0.898204,0.902256,0.952381,0.955696,0.889655,0.869863
0.30,0.924107,0.800643,0.883636,0.892734,0.813725,0.819048
0.35,0.891374,0.778626,0.831135,0.851948,0.805774,0.783505
0.40,0.867347,0.778626,0.831135,0.851948,0.805774,0.783505
0.45,0.846154,0.734021,0.814898,0.825893,0.792208,0.773913


In [40]:
def make_table(ai_group, human_group):
    agg_df = pd.DataFrame()
    agg_df["T"] = humand_df.index
    agg_df["T"] = agg_df["T"].astype(float).apply(lambda x: np.round(x, 2))
    agg_df["Ratio of Images Handled by AI"] = ratio_df[ai_group].values
    
    agg_df["AI Alone Performance"] = ai_df[ai_group].values
    agg_df["Human Performance on Remaining Images"] = humand_df[human_group].values
    agg_df["Aggregated Performance"] = (
        agg_df["Ratio of Images Handled by AI"] * agg_df["AI Alone Performance"]
    ) + (
        (1 - agg_df["Ratio of Images Handled by AI"])
        * agg_df["Human Performance on Remaining Images"]
    )

    agg_df = agg_df.set_index("T")

    agg_df["AI Alone Performance"] = (100 * agg_df["AI Alone Performance"]).round(2)
    agg_df["Human Performance on Remaining Images"] = (
        (100 * agg_df["Human Performance on Remaining Images"])
        .astype(float)
        .apply(lambda x: np.round(x, 2))
    )

    agg_df["Aggregated Performance"] = (
        (100 * agg_df["Aggregated Performance"])
        .astype(float)
        .apply(lambda x: np.round(x, 2))
    )

    agg_df["Ratio of Images Handled by AI"] = 100*agg_df[
        "Ratio of Images Handled by AI"
    ].round(4)

    return agg_df

In [41]:
method_tables = {}

for ai_group, human_group in zip(
    ["ResNet", "KNN", "EMD", "EMD", "CHM", "CHM"], ["ResNet-50", "kNN", "EMD-NN", "EMD-Corr", "CHM-NN", "CHM-Corr"]
):
    method_tables[human_group] = make_table(ai_group, human_group)


In [42]:
optimal_t

ResNet    0.45
KNN       0.45
EMD        0.4
CHM        0.5
dtype: object

In [43]:
method_tables['ResNet-50'].loc[[optimal_t['ResNet']]]


,Ratio of Images Handled by AI,AI Alone Performance,Human Performance on Remaining Images,Aggregated Performance
T,,,,
0.45,88.5,89.15,84.62,88.63


In [44]:
Aggregated_performance = {}

for ai_group, human_group in zip(
    ["ResNet", "KNN", "EMD", "EMD", "CHM", "CHM"], ["ResNet-50", "kNN", "EMD-NN", "EMD-Corr", "CHM-NN", "CHM-Corr"]
):
    Aggregated_performance[human_group] = method_tables[human_group].loc[[optimal_t[ai_group]]]['Aggregated Performance'].reset_index(drop=True)

Aggregated_performance_df = pd.DataFrame.from_dict(Aggregated_performance).T

In [45]:
method_tables['ResNet-50'].loc[[optimal_t['ResNet']]]


,Ratio of Images Handled by AI,AI Alone Performance,Human Performance on Remaining Images,Aggregated Performance
T,,,,
0.45,88.5,89.15,84.62,88.63


In [46]:
method_tables['kNN'].loc[[optimal_t['KNN']]]


,Ratio of Images Handled by AI,AI Alone Performance,Human Performance on Remaining Images,Aggregated Performance
T,,,,
0.45,86.34,89.44,73.4,87.24


In [47]:
ai_performance_df.round(2)

,AI Performance
ResNet,86.11
KNN,85.95
EMD,85.91
CHM,85.36


In [48]:
# remove index
Aggregated_performance_df

,0
ResNet-50,88.63
kNN,87.24
EMD-NN,87.80
EMD-Corr,88.02
CHM-NN,88.05
CHM-Corr,87.89


In [49]:
# combining two df
# rename index to match the names
ai_performance_df.index = ['ResNet-50', 'kNN', 'EMD-Corr', 'CHM-Corr']

combined_table = pd.concat([ai_performance_df.round(2), Aggregated_performance_df, ], axis=1)
combined_table.columns = ['AI Alone Performance', 'Human-AI Performance']

# calculate the delta
combined_table['Improvement'] = combined_table['Human-AI Performance'] - combined_table['AI Alone Performance']


In [50]:
combined_table

,AI Alone Performance,Human-AI Performance,Improvement
ResNet-50,86.11,88.63,2.52
kNN,85.95,87.24,1.29
EMD-Corr,85.91,88.02,2.11
CHM-Corr,85.36,87.89,2.53
EMD-NN,NaN,87.80,NaN
CHM-NN,NaN,88.05,NaN


In [51]:
# drop last two rows
combined_table = combined_table.drop(['EMD-NN', 'CHM-NN'])

In [52]:
combined_table

,AI Alone Performance,Human-AI Performance,Improvement
ResNet-50,86.11,88.63,2.52
kNN,85.95,87.24,1.29
EMD-Corr,85.91,88.02,2.11
CHM-Corr,85.36,87.89,2.53


In [53]:
# Add a row for calculating the average
combined_table.loc['Average'] = combined_table.mean()

In [54]:
combined_table.round(2)

,AI Alone Performance,Human-AI Performance,Improvement
ResNet-50,86.11,88.63,2.52
kNN,85.95,87.24,1.29
EMD-Corr,85.91,88.02,2.11
CHM-Corr,85.36,87.89,2.53
Average,85.83,87.94,2.11
